Adapted from Mathilde Koch, INRAe, Faulon's group

The aim of this notebook is to generate the initial training set (or initial plate) for the subsequent active learning loop.

# Python libraries imports

In [1]:
import numpy as np
import random

# Create a folder to save the resulting dataset

In [43]:
data_folder = "echo_files"

**Important**  
The seed was not set when the file was first executed.  
The concentrations used for the initial plate in the paper can be found in **data/plate_AL_1_raw_everything.csv**.

**Caution**  
When you run the code under the pre-existing files architecture, the results are overwritten in the **initial_plate.csv** dataset.

# Defining the tested concentrations and volumes

The concentrations below are defined as a proportion of the maximum allowed concentration.

In [3]:
# Allowed concentration ratios for each component. 
# Mostly, four ratios were chosen for each component.

mg_gluta_conc = [0.1, 0.3, 0.5, 1]
K_gluta_conc = [0.1, 0.3, 0.5, 1]
aa_conc = [0.1, 0.3, 0.5, 1]
trna_conc = [0.1, 0.3, 0.5, 1]
coa_conc = [0.1, 0.3, 0.5, 1]
nad_conc = [0.1, 0.3, 0.5, 1]
camp_conc = [0.1, 0.3, 0.5, 1]
folinic_acid_conc = [0.1, 0.3, 0.5, 1]
spermidine_conc = [0.1, 0.3, 0.5, 1]
pga_conc = [0.1, 0.3, 0.5, 1]
nucleo_conc = [0.1, 0.3, 0.5, 1]
DNA_mix = [0, 1]

peg_conc = [1]
hepes_conc = [1]
extract_conc = [1]
promoter_conc = [1]
RBS_conc = [1]

In [4]:
# Maximum concentration for each component. 
# The resulting concentration is determined as follows: Maximum concentration * Ratio.

mg_gluta_max = 4
K_gluta_max = 80
aa_max = 1.5
trna_max = 0.2
coa_max = 0.26
nad_max = 0.33
camp_max = 0.75
folinic_acid_max = 0.068
spemidine_max = 1
pga_max = 30
nucleo_mix_max = 1.5
DNA_max = 50

peg_max = 2
hepes_max = 50
extract_max = 30
promoter_max = 10
RBS_max = 10

In [5]:
# Volume required and stocks - calculated for compatibility with 2.5 nL nanodrops of the "Echo liquid handler".

vol_max = 10.5
n = 5  # Number of drops at minimal concentration. We chose to have at least 5 drops even at minimal concentration.

stock_extract = 89.5
stock_mg_gluta = 168
stock_K_gluta = 3360
stock_aa = 6
stock_peg = 40
stock_hepes = 2100
stock_trna = 84/n
stock_coa = 109.2/n
stock_nad = 138.6/n
stock_camp = 315/n
stock_folinic_acid = 28.56/n
stock_spermidine = 420/n
stock_pga = 1400
stock_nucleo = 630/n
stock_dna = 300

# Defining control samples (maximum and autofluorescence)

This defines the control samples (reference) used in the first plate, that have maximum concentrations for all components. 
The autofluorescence **<sup>1</sup>** control has no DNA.

**<sup>1</sup>** is the fluorescence measured in the cell-free reaction supplemented with water and using the reference composition.

In [6]:
# Creating reference extract where all components are at the maximum/highest concentrations.

maximum_extract = np.ones((1, 17), dtype = 'int')

max_sample_conc = [maximum_extract[0, 0] * extract_max,
                   maximum_extract[0, 1] * mg_gluta_max, 
                   maximum_extract[0, 2] * K_gluta_max, 
                   maximum_extract[0, 3] * aa_max, 
                   maximum_extract[0, 4] * peg_max, 
                   maximum_extract[0, 5] * hepes_max, 
                   maximum_extract[0, 6] * trna_max, 
                   maximum_extract[0, 7] * coa_max, 
                   maximum_extract[0, 8] * nad_max, 
                   maximum_extract[0, 9] * camp_max, 
                   maximum_extract[0, 10] * folinic_acid_max,
                   maximum_extract[0, 11] * spemidine_max, 
                   maximum_extract[0, 12] * pga_max, 
                   maximum_extract[0, 13] * nucleo_mix_max,
                   maximum_extract[0, 14] * DNA_max,
                   maximum_extract[0, 15] * promoter_max,
                   maximum_extract[0, 16] * RBS_max
                  ]

In [7]:
# Creating reference autofluorescence extract 

autofluorescence_extract = np.ones((1, 17), dtype = 'int')

autofluorescence_sample_conc = [autofluorescence_extract[0, 0] * extract_max,
                               autofluorescence_extract[0, 1] * mg_gluta_max, 
                               autofluorescence_extract[0, 2] * K_gluta_max, 
                               autofluorescence_extract[0, 3] * aa_max, 
                               autofluorescence_extract[0, 4] * peg_max, 
                               autofluorescence_extract[0, 5] * hepes_max, 
                               autofluorescence_extract[0, 6] * trna_max, 
                               autofluorescence_extract[0, 7] * coa_max, 
                               autofluorescence_extract[0, 8] * nad_max, 
                               autofluorescence_extract[0, 9] * camp_max, 
                               autofluorescence_extract[0, 10] * folinic_acid_max,
                               autofluorescence_extract[0, 11] * spemidine_max, 
                               autofluorescence_extract[0, 12] * pga_max, 
                               autofluorescence_extract[0, 13] * nucleo_mix_max,
                               autofluorescence_extract[0, 14] * 0, # No DNA in the autofluorescence control
                               autofluorescence_extract[0, 15] * promoter_max,
                               autofluorescence_extract[0, 16] * RBS_max
                               ]

In [8]:
max_sample_conc = np.reshape(max_sample_conc, (1, 17))

autofluorescence_sample_conc = np.asarray(autofluorescence_sample_conc)
autofluorescence_sample_conc = np.reshape(autofluorescence_sample_conc, (1, 17))

ctrl_array = max_sample_conc
ctrl_array = np.concatenate((ctrl_array, autofluorescence_sample_conc), axis = 0)

# Defining all max, except one low, plus max 

This defines a set of combinations we wanted to test on the initial plate.  
* Here, all concentrations are at the maximum possible, except one that is at the lowest value. This therefore makes 11 forced combinations. 
* We also added the combination with everything at the maximum, as well as one more point for NTP effects.

In [9]:
all_max_except_one_low = max_sample_conc

In [10]:
extract = extract_conc[0]
peg = peg_conc[0]
hepes = hepes_conc[0]
dna = DNA_mix[1]
promoter = promoter_conc[0]
RBS = RBS_conc[0]

mg_gluta = mg_gluta_conc[-1]
K_gluta = K_gluta_conc[-1]
aa = aa_conc[-1]
trna = trna_conc[-1]
coa = coa_conc[-1]
nad = nad_conc[-1]
camp = camp_conc[-1]
folinic_acid = folinic_acid_conc[-1]
spermidine = spermidine_conc[-1]
pga = pga_conc[-1]
nucleo = nucleo_conc[-1]

In [11]:
this_sample_conc = [extract * extract_max,
                   0.1 * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max, 
                   hepes * hepes_max,  
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,  
                   promoter * promoter_max,  
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [12]:
this_sample_conc = [extract * extract_max, 
                   mg_gluta * mg_gluta_max, 
                   0.1 * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,   
                   hepes * hepes_max, 
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max, 
                   promoter * promoter_max,  
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [12]:
this_sample_conc = [extract * extract_max,  
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   0.1 * aa_max, 
                   peg * peg_max,    
                   hepes * hepes_max,  
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,  
                   promoter * promoter_max,  
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [13]:
this_sample_conc = [extract * extract_max,  
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,    
                   hepes * hepes_max, 
                   0.1 * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,  
                   promoter * promoter_max,  
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [14]:
this_sample_conc = [extract * extract_max,  
                  mg_gluta* mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,    
                   hepes * hepes_max,   
                   trna * trna_max, 
                   0.1 * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max, 
                   promoter * promoter_max,  
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [15]:
this_sample_conc = [extract * extract_max, 
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,    
                   hepes * hepes_max,  
                   trna * trna_max, 
                   coa * coa_max, 
                   0.1 * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,  
                   promoter * promoter_max,
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [16]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,   
                   hepes * hepes_max,   
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   0.1 * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max, 
                   promoter * promoter_max, 
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [17]:
this_sample_conc = [extract * extract_max, 
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,   
                   hepes * hepes_max, 
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   0.1 * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max, 
                   promoter * promoter_max, 
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [18]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,   
                   hepes * hepes_max, 
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   0.1 * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max, 
                   promoter * promoter_max,  
                   RBS * RBS_max] 

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [19]:
this_sample_conc = [extract * extract_max, 
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,   
                   hepes * hepes_max,   
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   0.1 * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,  
                   promoter * promoter_max,  
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [20]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,   
                   hepes * hepes_max,  
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   0.1 * nucleo_mix_max,
                   dna * DNA_max,  
                   promoter * promoter_max,
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [21]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,   
                   hepes * hepes_max,  
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   0.3 * nucleo_mix_max,
                   dna * DNA_max,  
                   promoter * promoter_max,  
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_max_except_one_low = np.concatenate((all_max_except_one_low, this_sample_conc), axis = 0)

In [22]:
print(all_max_except_one_low.shape)

(12, 17)


# Defining all min except one high plus min 

This defines a set of combinations we wanted to test on the initial plate. Here, all concentrations are at the minimum possible, except one that is at the maximum values. This therefore makes 11 forced combinations. We also forced the combination with all reagents at the lowest values.

In [23]:
extract = extract_conc[0]
peg = peg_conc[0]
hepes = hepes_conc[0]
dna = DNA_mix[1]
promoter = promoter_conc[0]
RBS = RBS_conc[0]

mg_gluta = mg_gluta_conc[0]
K_gluta = K_gluta_conc[0]
aa = aa_conc[0]
trna = trna_conc[0]
coa = coa_conc[0]
nad = nad_conc[0]
camp = camp_conc[0]
folinic_acid = folinic_acid_conc[0]
spermidine = spermidine_conc[0]
pga = pga_conc[0]
nucleo = nucleo_conc[0]

In [24]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,  
                   hepes * hepes_max,
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max, 
                   promoter * promoter_max,
                   RBS * RBS_max] 

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = this_sample_conc

In [25]:
this_sample_conc = [extract * extract_max,
                   1 * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max, 
                   hepes * hepes_max,
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max, 
                   promoter * promoter_max, 
                   RBS * RBS_max] 

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [26]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   1 * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,  
                   hepes * hepes_max,
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,
                   promoter * promoter_max,
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [27]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   1 * aa_max, 
                   peg * peg_max, 
                   hepes * hepes_max,  
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max, 
                   promoter * promoter_max, 
                   RBS * RBS_max]  

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [28]:
this_sample_conc = [extract * extract_max, 
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,  
                   hepes * hepes_max,
                   1 * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,
                   promoter * promoter_max,
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [29]:
this_sample_conc = [extract * extract_max,
                  mg_gluta* mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max, 
                   hepes * hepes_max,  
                   trna * trna_max, 
                   1 * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,
                   promoter * promoter_max, 
                   RBS * RBS_max] 

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [30]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max, 
                   hepes * hepes_max,  
                   trna * trna_max, 
                   coa * coa_max, 
                   1 * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,
                   promoter * promoter_max,
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [31]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,
                   hepes * hepes_max,
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   1 * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,
                   promoter * promoter_max, 
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [32]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max, 
                   hepes * hepes_max,
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   1 * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,
                   promoter * promoter_max,
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [33]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,
                   hepes * hepes_max,
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   1 * spemidine_max, 
                   pga * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,
                   promoter * promoter_max,
                   RBS * RBS_max] 

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [34]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,
                   hepes * hepes_max,
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   1 * pga_max,
                   nucleo * nucleo_mix_max,
                   dna * DNA_max,
                   promoter * promoter_max,
                   RBS * RBS_max]

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [35]:
this_sample_conc = [extract * extract_max,
                   mg_gluta * mg_gluta_max, 
                   K_gluta * K_gluta_max, 
                   aa * aa_max, 
                   peg * peg_max,  
                   hepes * hepes_max, 
                   trna * trna_max, 
                   coa * coa_max, 
                   nad * nad_max,
                   camp * camp_max,
                   folinic_acid * folinic_acid_max,
                   spermidine * spemidine_max, 
                   pga * pga_max,
                   1 * nucleo_mix_max,
                   dna * DNA_max,
                   promoter * promoter_max,
                   RBS * RBS_max] 

this_sample_conc = np.reshape(this_sample_conc, (1, 17)) 
all_min_except_high = np.concatenate((all_min_except_high, this_sample_conc), axis = 0)

In [36]:
print(all_min_except_high.shape)

(12, 17)


# Filling the rest randomly

The rest of the plate will be filled randomly, while avoiding repeats.

In [37]:
answer_array = np.concatenate((ctrl_array, all_max_except_one_low, all_min_except_high), axis = 0)

In [38]:
print(answer_array.shape)
# The expected shape is (27,17)

(26, 17)


In [39]:
def present_in_array_index(new_sample, array):
    
    """
    Verify if a sample is present in an array.
    """
    
    if array is None:
        return(False, -1)
    present = False
    new_sample = np.reshape(np.array(new_sample), (1,17))
    for i in range(array.shape[0]):
        if np.array_equiv(array[i,:],new_sample):
            present = True
            break   
    return(present, i)

In [40]:
def generate_random_grid(array_to_avoid, sample_size = 100):
    active_learning_array = array_to_avoid
    if array_to_avoid is None:
        answerSize = 0
    else:
        answerSize = active_learning_array.shape[0]
    
    # Reagents not allowed to vary
    extract = extract_conc[0]
    peg = peg_conc[0]
    hepes = hepes_conc[0]
    dna = DNA_mix[1]
    promoter = promoter_conc[0]
    RBS = RBS_conc[0]

    while answerSize < sample_size:
        this_sample = np.random.randint(0, high=3, size=11, dtype='int')
        this_sample_conc = [extract * extract_max,
                            mg_gluta_conc[this_sample[0]] * mg_gluta_max, 
                            K_gluta_conc[this_sample[1]] * K_gluta_max, 
                            aa_conc[this_sample[2]] * aa_max, 
                            peg * peg_max, 
                            hepes * hepes_max, 
                            trna_conc[this_sample[3]] * trna_max, 
                            coa_conc[this_sample[4]] * coa_max, 
                            nad_conc[this_sample[5]] * nad_max, 
                            camp_conc[this_sample[6]] * camp_max, 
                            folinic_acid_conc[this_sample[7]] * folinic_acid_max,
                            spermidine_conc[this_sample[8]] * spemidine_max, 
                            pga_conc[this_sample[9]] * pga_max, 
                            nucleo_conc[this_sample[10]] * nucleo_mix_max,
                            dna * DNA_max,  # not modified
                            promoter * promoter_max,  # not modified
                            RBS * RBS_max]
        
        this_sample_conc = np.reshape(this_sample_conc, (1, 17))
        if not present_in_array_index(this_sample_conc, active_learning_array)[0]:
            answerSize = answerSize + 1
            if active_learning_array is None:
                active_learning_array = this_sample_conc
            else:
                active_learning_array = np.concatenate((active_learning_array, this_sample_conc), axis = 0)
    return(active_learning_array)

In [41]:
answer_array = generate_random_grid(answer_array, sample_size = 102)

In [44]:
# Saving the data

headers = "extract; mg_gluta; K_gluta; aa; peg; hepes; trna; coa; nad; camp; folinic_acid; spemidin; pga; nucleo_mix; DNA; promoter; RBS"

np.savetxt("{}/initial_plate.csv".format(data_folder), answer_array, delimiter=";", header = headers,fmt='%.5f')